In [1]:
import fasttext
import pandas as pd
import numpy as np
import re

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.float_format', '{:,.8f}'.format)
pd.set_option('display.min_rows', 6)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

##### Обучим модель для классификации чеков. В качестве разметки возьмем каталог wildberries.


In [2]:
wb = pd.read_parquet('./WB.parquet')

In [3]:
wb.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4921161 entries, 0 to 4929045
Data columns (total 8 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   category_1  object
 1   category_2  object
 2   category_3  object
 3   category_4  object
 4   category_5  object
 5   name        object
 6   brand       object
 7   supplier    object
dtypes: object(8)
memory usage: 337.9+ MB


In [4]:
wb.sample(5)

,category_1,category_2,category_3,category_4,category_5,name,brand,supplier
index,,,,,,,,
3560025,Спорт,Единоборства,ММА,None,None,Рашгард футболка с коротким рукавом для единоб...,Cody Lundin,СПОРТ
1567422,Дом,Отдых на природе,Мангалы и коптильни,None,None,Шампура с деревянной ручкой набор для шашлыка ...,ШАМПУР-ЮГ,ШАМПУР-ЮГ
1672367,Дом,Хозяйственные товары,Товары для уборки,Чистящие средства,None,"Чистящие средства для стеклокерамики AZELIT 0,6кг",GRASS,Белвес
816364,Мужчинам,Подарки мужчинам,Белье,None,None,Носки цветные с принтом и рисунком,MarusyaSocks,MARUSYA прикольные носки с принтами для всей с...
4387364,Для ремонта,Лакокрасочные материалы,Покрытия для дерева,None,None,Масло для дерева Deco Oil Terrace ОрехГварнери...,Kraskovar,ЛидерГрупп Строй


In [5]:
wb['category_1'].unique()

array(['Женщинам', 'Обувь', 'Детям', 'Мужчинам', 'Дом', 'Красота',
       'Аксессуары', 'Электроника', 'Игрушки', 'Мебель',
       'Товары для взрослых', 'Продукты', 'Бытовая техника', 'Зоотовары',
       'Спорт', 'Автотовары', 'Транспортные средства', 'Книги',
       'Ювелирные изделия', 'Для ремонта', 'Сад и дача', 'Здоровье',
       'Канцтовары', 'Культурный код'], dtype=object)

In [6]:
# Немного подкорректируем разметку
wb['category_1'][wb['category_1'].isin(['Женщинам', 'Мужчинам'])] = 'Одежда'

In [7]:
train_ds = wb[['category_1', 'name']]
train_ds = train_ds.drop_duplicates()

In [8]:
train_ds.sample(5)

,category_1,name
index,,
3886312,Автотовары,Ручка КПП в стиле Весты ВАЗ-2108-21099
3507358,Спорт,Сывороточный протеин Ultra Whey 300г - Фисташк...
2763833,Мебель,"Кронштейны для полки ""Элимет"" 220 мм, цвет бел..."
4357519,Для ремонта,"Изолятор для монтажа ретро проводки, цвет песо..."
1831005,Красота,Помада для губ матовая жидкая увлажняющая № 03


In [9]:
# уберем ненужные данные из имен
train_ds['name'] = train_ds['name'].str.lower()
train_ds['name'] = train_ds['name'].str.replace('[^a-zа-яё]+', ' ', regex=True, flags=re.IGNORECASE)
train_ds['name'] = train_ds['name'].str.replace(r'\b[a-zа-яё]{,2}\b', '', regex=True, flags=re.IGNORECASE)
train_ds['name'] = train_ds['name'].str.replace(' {2,}', ' ', regex=True, flags=re.IGNORECASE)
train_ds['name'] = train_ds['name'].str.strip()

In [10]:
train_ds.sample(5)

,category_1,name
index,,
408552,Обувь,сумка для обуви молнии
4324460,Для ремонта,перчатки рабочие строительные нитей пвх
4906984,Канцтовары,бумага цветная листов пастель зелёная
4751461,Сад и дача,садовая мебель комплект для дачи
4914601,Канцтовары,электрическая точилка для карандашей


In [11]:
# создадим мепер
mapper_ = train_ds['category_1'].drop_duplicates().sort_values().reset_index(drop=True).to_dict()
mapper = {v: k for k, v in mapper_.items()}
train_ds['label'] = train_ds['category_1'].map(mapper)

In [12]:
# соберем строку для для датасета
train_ds['train_row'] = '__label__' + train_ds['label'].astype(str) + ' ' + train_ds['name'] + '\n'

In [13]:
train_ds.sample(5)

,category_1,name,label,train_row
index,,,,
1999774,Красота,влажные салфетки детские штук,11,__label__11 влажные салфетки детские штук\n
835197,Одежда,набор коллекционных монет года,15,__label__15 набор коллекционных монет года\n
919325,Дом,кружка тебя всё получится,5,__label__5 кружка тебя всё получится\n
2992574,Бытовая техника,насадки фен для волос supersonic super hair dryer,2,__label__2 насадки фен для волос supersonic su...
4802713,Здоровье,витамин капсул,6,__label__6 витамин капсул\n


In [14]:
# перемешаем датасета
train_ds = train_ds.sample(frac=1)

In [15]:
# запишем датасет
with open('train_ds.txt', mode='w') as file:
    file.writelines(train_ds['train_row'].values)

In [16]:
# обучим модель
model = fasttext.train_supervised(input='train_ds.txt', lr=0.5, dim=300, 
                          epoch=100, wordNgrams=3, loss='hs', bucket=5000000)

Read 16M words
Number of words:  273241
Number of labels: 23
Progress: 100.0% words/sec/thread:  642478 lr:  0.000000 avg.loss:  0.121366 ETA:   0h 0m 0s 65.3% words/sec/thread:  637659 lr:  0.173739 avg.loss:  0.156928 ETA:   0h 4m56s 73.7% words/sec/thread:  639004 lr:  0.131507 avg.loss:  0.145848 ETA:   0h 3m44s


In [22]:
model.test('train_ds.txt')

(2483055, 0.9061426347785289, 0.9061426347785289)

In [17]:
# Возьмем дата фрейм с чеками
df = pd.read_csv('unsupervised_dataset.csv')

In [18]:
df.head()

,id,name
0,0,Зубная щетка Орал Би Три эффект Деликатное отб...
1,1,салфетки VISTER влажные для ко
2,2,Платье женское DR8517K 7Л8999 Светло-серый 449...
3,3,"ЛАКОМСТВО ""ДЕРЕВЕНСКИЕ ЛАКОМСТВА"" д/собак мини..."
4,4,Суппорт гитарный Ergo Play Troster


In [21]:
# Выполним субъективную оценки модели
tdf = df.sample(10)
tdf['name_c'] = tdf['name'].str.lower()
tdf['name_c'] = tdf['name_c'].str.replace('[^a-zа-яё]+', ' ', regex=True, flags=re.IGNORECASE)
tdf['name_c'] = tdf['name_c'].str.replace(r'\b[a-zа-яё]{,2}\b', '', regex=True, flags=re.IGNORECASE)
tdf['name_c'] = tdf['name_c'].str.replace(' {2,}', ' ', regex=True, flags=re.IGNORECASE)
tdf['name_c'] = tdf['name_c'].str.strip()
tdf['pred'] = model.predict([*tdf['name_c']])[0]
tdf['pred'] = tdf['pred'].str[0].str.replace('__label__', '').astype('int').map(mapper_)
tdf

,id,name,name_c,pred
965619,965619,"(БЛ) Котлета из курицы, 088 гр.",котлета курицы,Продукты
308482,308482,3ST CHOICE корм д/с средних и крупных пород 7к...,choice корм средних крупных пород курица,Зоотовары
547136,547136,Свитшот Acoola REP22700531,свитшот acoola rep,Детям
873611,873611,"Жилет мужской утепленный ""Рабочий""",жилет мужской утепленный рабочий,Одежда
645675,645675,КАЛЬЦИЯ ГЛЮКОНАТ Р-Р ДЛЯ В/В И В/М ВВ 765МГ/МЛ...,кальция глюконат для амп,Здоровье
575030,575030,"CLUB7, КОЗЫРЁКСОЛНЦЕЗАЩИТНЫЙНАРЕЗИНКЕ (JQO01, ...",club козырёксолнцезащитныйнарезинке jqo,Игрушки
867044,867044,ШС70 (Крем д/обуви ДАМАВИК жест. банка черный ...,крем обуви дамавик жест банка черный,Красота
811570,811570,4646309909833 Футболка жен. Принт,футболка жен принт,Детям
194542,194542,"ГОРШОК ОБРУЧ №7 ГЛЯНЕЦ БЕЖ (8,8л)",горшок обруч глянец беж,Сад и дача
786305,786305,Сельдь ф/кусочки в майонез. заливке 321 гр Мар...,сельдь кусочки майонез заливке марти,Продукты


#### Вывод: и так сойдет.